In [3]:
from diffusion_planner.data_process.data_processor import DataProcessor
from nuplan.common.actor_state.state_representation import Point2D
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor

from diffusion_planner.data_process.utils import (
get_scenario_map,
get_filter_parameters,
sampled_tracked_objects_to_tensor,
)

from diffusion_planner.data_process.roadblock_utils import (
  route_roadblock_correction
)


from diffusion_planner.data_process.agent_process import(sampled_tracked_objects_to_array_list,
                                                         sampled_static_objects_to_array_list,
                                                         agent_past_process,)
from diffusion_planner.data_process.map_process import(map_process,
                                                       get_neighbor_vector_set_map)
import numpy as np
import torch

In [5]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # mini db files

map_path = "/share/data_cold/open_data/nuplan/maps"
save_path = "/cailiu2/Diffusion-Planner/data/processed"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"
total_scenarios = 10
scenario_mapping = ScenarioMapping(scenario_map=get_scenario_map(), subsample_ratio_override=0.5)
builder = NuPlanScenarioBuilder(db_path, map_path, None, None, map_version, scenario_mapping = scenario_mapping)
worker = SingleMachineParallelExecutor(use_process_pool=True)
scenario_filter = ScenarioFilter(*get_filter_parameters(num_scenarios_per_type=30000,
                                                          limit_total_scenarios=total_scenarios))
scenarios = builder.get_scenarios(scenario_filter, worker)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
del worker, builder, scenario_filter
processor = DataProcessor(scenarios,device)
# processor.work(save_path, debug=False)

Worker threads: 128


In [6]:
from diffusion_planner.data_process.utils import convert_absolute_quantities_to_relative,_global_state_se2_array_to_local,_global_velocity_to_local
from nuplan.planning.training.preprocessing.utils.agents_preprocessing import EgoInternalIndex, AgentInternalIndex


In [92]:
from nuplan.common.actor_state.tracked_objects_types import TrackedObjectType
scenario = processor._scenarios[0]
map_name = scenario._map_name
token = scenario.token
processor.scenario = scenario
processor.map_api = scenario.map_api

"""
ego
"""
ego_state = processor.scenario.initial_ego_state
ego_coords = Point2D(ego_state.rear_axle.x, ego_state.rear_axle.y)
anchor_ego_state = np.array([ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading], \
                            dtype=np.float64)

"""
neighbor agents
"""
neighbors = processor.scenario.initial_tracked_objects
neighbor_past = list(processor.scenario.get_past_tracked_objects(iteration= 0, time_horizon=1.0))
tracked_objects, tracked_objects_types, tracked_obj_tokens = sampled_tracked_objects_to_tensor(neighbors.tracked_objects)
#  tracked_objects_past, _ = sampled_tracked_objects_to_array_list(neighbor_past)
static_objects, static_objects_types =sampled_static_objects_to_array_list(neighbor_past[-1])
tracked_objects_list = []
for tracked_object in tracked_objects:
  tracked_objects_list.append(tracked_object)
_, neighbor_past, selected_indices, static_objects = agent_past_process(None, tracked_objects, tracked_objects_types, processor.num_agents,\
                                                        static_objects, static_objects_types, \
                                                        processor.num_static, processor.max_ped_bike, anchor_ego_state)
# print(selected_indices[12])

objects_tokens = [tracked_obj_tokens[key] for key in selected_indices]
# print(len(selected_indices))
# print(len(objects_tokens))
# print(objects_tokens)
neighbor_future = list(processor.scenario.get_future_tracked_objects(iteration= 0, time_horizon=8.0, num_samples =int(8*10)))
# print(len(neighbor_future))
slected_objs_future_traj = np.full((len(neighbor_future), len(objects_tokens), 11), np.nan, dtype=np.float64)
for frame_id in range(len(neighbor_future)):
  tracked_objects = neighbor_future[frame_id]
  all_future_neighbor_info = []
  current_frame_tracked_objects = {tracked_object.track_token: tracked_object for tracked_object in tracked_objects.tracked_objects}
  for i in range(len(objects_tokens)):
    if objects_tokens[i] in current_frame_tracked_objects.keys():
      agent_state = current_frame_tracked_objects[objects_tokens[i]]

      agent_global_poses = np.array([[agent_state.center.x, agent_state.center.y, agent_state.center.heading]])
      agent_global_velocities = np.array([[agent_state.velocity.x, agent_state.velocity.y]])
      transformed_poses = _global_state_se2_array_to_local(agent_global_poses, anchor_ego_state)
      transformed_velocities = _global_velocity_to_local(agent_global_velocities, anchor_ego_state[-1])
      local_agent_state = np.zeros((1, 11))
      local_agent_state[:, 0] = transformed_poses[:, 0]
      local_agent_state[:, 1] = transformed_poses[:, 1]
      local_agent_state[:, 2] = np.cos(transformed_poses[:, 2])
      local_agent_state[:, 3] = np.sin(transformed_poses[:, 2])
      local_agent_state[:, 4] = transformed_velocities[:, 0]
      local_agent_state[:, 5] = transformed_velocities[:, 1]
      local_agent_state[:, 6] = agent_state.box.width
      local_agent_state[:, 7] = agent_state.box.length
      if agent_state.tracked_object_type == TrackedObjectType.VEHICLE:
        local_agent_state[:, 8:] = [1, 0, 0]  # Mark as VEHICLE
      elif agent_state.tracked_object_type == TrackedObjectType.PEDESTRIAN:
        local_agent_state[:, 8:] = [0, 1, 0]  # Mark as PEDESTRIAN
      else:  # TrackedObjectType.BICYCLE
        local_agent_state[:, 8:] = [0, 0, 1]  # Mark as BICYCLE
      slected_objs_future_traj[frame_id, i, :] = local_agent_state
# selected_neighbor_future = np.full(len(objects_tokens))
slected_objs_future_traj = slected_objs_future_traj.transpose(1, 0, 2)
print(neighbor_past.shape)
print(slected_objs_future_traj.shape)
# neighbor_future.tracked_objects
# for token in objects_tokens:
#   selected_neighbor_future.append([selected_tracked_object for selected_tracked_object in neighbor_future \
#                                   if selected_tracked_object.tracked_objects.track_token == token])

(30, 1, 11)
(30, 80, 11)


In [93]:
future_ego_states = processor.scenario.get_ego_future_trajectory(iteration=0, num_samples=int(8*10), time_horizon= 8.0)
future_ego_states_numpy = np.array([(ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading, \
                  ego_state.dynamic_car_state.rear_axle_velocity_2d.x, ego_state.dynamic_car_state.rear_axle_velocity_2d.y,\
                  ego_state.dynamic_car_state.rear_axle_acceleration_2d.x, ego_state.dynamic_car_state.rear_axle_acceleration_2d.y ) for ego_state in future_ego_states], dtype = np.float64)
local_future_ego_states = convert_absolute_quantities_to_relative(future_ego_states_numpy, anchor_ego_state,'ego')

ego_state_array = np.array([(ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading, \
                  ego_state.dynamic_car_state.rear_axle_velocity_2d.x, ego_state.dynamic_car_state.rear_axle_velocity_2d.y,\
                  ego_state.dynamic_car_state.rear_axle_acceleration_2d.x, ego_state.dynamic_car_state.rear_axle_acceleration_2d.y )], dtype = np.float64)
ego_current_states = convert_absolute_quantities_to_relative(ego_state_array,anchor_ego_state,'ego' )
future_ego_states_numpy =np.concatenate((ego_current_states,future_ego_states_numpy),axis = 0)
future_ego_traj_noise = np.concatenate((
    future_ego_states_numpy[:, :2],  # 取前两列 (x, y)
    np.cos(future_ego_states_numpy[:, 3:4]),  # 取第三列 (heading) 并计算 cos
    np.sin(future_ego_states_numpy[:, 3:4])   # 取第三列 (heading) 并计算 sin
), axis=1)
print(ego_state_array)
print(future_ego_states_numpy.shape)
print(future_ego_traj_noise.shape)


[[ 0.00000000e+00  0.00000000e+00  2.27889064e-17 -2.33842428e-03
   2.39401828e-05  1.25967396e-02  1.27703869e-02]]
(81, 7)
(81, 4)


In [94]:
from diffusion_planner.model.diffusion_utils.sde import SDE, VPSDE_linear
def add_agent_traj_noise(origin_traj : np.array, sde:SDE, device):
    P, T, D = origin_traj.shape # P: agent num， T: frame nums, D: dim
    assert D == 4  #x, y, cos, sin
    origin_traj_tensor = torch.tensor(origin_traj, dtype=torch.float32).to(device)

    t = torch.rand(origin_traj.shape[0])
    mean, std = sde.marginal_prob(origin_traj_tensor, t)

    noisy_traj = torch.zeros_like(mean)
    noisy_traj = mean + std*noisy_traj
    theta_norm = torch.sqrt(noisy_traj[:,:,2]**2 + noisy_traj[:,:,3]**2 + 1e-6)
    noisy_traj[:,:,2] /= theta_norm
    noisy_traj[:,:,3] /= theta_norm

    return noisy_traj, t #tensor

In [96]:
future_ego_states_numpy = np.array([future_ego_traj_noise])
print(future_ego_states_numpy.shape)
sde1 = VPSDE_linear()

future_ego_noisy_traj,t  = add_agent_traj_noise(future_ego_states_numpy, sde1, device = 'cpu')
print(future_ego_noisy_traj.shape)


(1, 81, 4)
torch.Size([1, 81, 4])


In [ ]:
# print(type(local_future_ego_states))
# print(local_future_ego_states.shape)
# print(local_future_ego_states[0][:2])

print(neighbor_past.shape)
print(slected_objs_future_traj.shape)
slected_objs_future_traj= np.concatenate((neighbor_past, slected_objs_future_traj), axis = 1)

# print(slected_objs_future_traj[2, :, :2])
# print(slected_objs_future_traj[29, :, :2])
# # print(slected_objs_future_traj[10, 1, :2])
# # print(slected_objs_future_traj[10, 2, :2])

(30, 1, 11)
(30, 80, 11)
(30, 81, 11)
[[ 4.48345613 -4.55687094]
 [ 4.46854536 -4.54409703]
 [ 4.45589865 -4.55093455]
 [ 4.44309554 -4.55699446]
 [ 4.43011199 -4.56031671]
 [ 4.41907081 -4.56324914]
 [ 4.41139103 -4.56506833]
 [ 4.4052394  -4.56671293]
 [ 4.40045676 -4.56853253]
 [ 4.39616283 -4.57039844]
 [ 4.39325585 -4.57148423]
 [ 4.39151903 -4.57212056]
 [ 4.39063017 -4.5723799 ]
 [ 4.38745016 -4.57131092]
 [ 4.3856349  -4.57088705]
 [ 4.38516369 -4.5711056 ]
 [ 4.38496023 -4.57033664]
 [ 4.38589158 -4.56975588]
 [ 4.38632612 -4.56907126]
 [ 4.38586368 -4.56768715]
 [ 4.38445494 -4.56639898]
 [ 4.38078201 -4.56692012]
 [ 4.37635748 -4.56969109]
 [ 4.37065359 -4.57008436]
 [ 4.36396836 -4.56795677]
 [ 4.35595086 -4.56364245]
 [ 4.34614005 -4.55661575]
 [ 4.33506659 -4.54842501]
 [ 4.32288058 -4.53225092]
 [ 4.31196398 -4.50479416]
 [ 4.29989814 -4.46838338]
 [ 4.28769065 -4.42881649]
 [ 4.27495557 -4.39013071]
 [ 4.26232081 -4.340349  ]
 [ 4.25227572 -4.29786063]
 [ 4.24637964 -4.

In [102]:
print(slected_objs_future_traj.shape)
slected_objs_future_traj = np.concatenate((
    slected_objs_future_traj[:,:, :2],  # 取前两列 (x, y)
    np.cos(slected_objs_future_traj[:,:, 3:4]),  # 取第三列 (heading) 并计算 cos
    np.sin(slected_objs_future_traj[:,:, 3:4])   # 取第三列 (heading) 并计算 sin
), axis=2)
print(slected_objs_future_traj.shape)

(30, 81, 11)
(30, 81, 4)


In [104]:
future_agent_noisy_traj,t1  = add_agent_traj_noise(slected_objs_future_traj, sde1, device = 'cpu')
print(future_agent_noisy_traj.shape)
print(t1.shape)

torch.Size([30, 81, 4])
torch.Size([30])


In [110]:
result = torch.concat((future_ego_noisy_traj,future_agent_noisy_traj),dim=0 )
result_t = torch.concat((t, t1), dim = 0)
print(result_t.shape)

torch.Size([31])


In [26]:
import os

# 处理时间格式

save_path = "/cailiu2/Diffusion-Planner/data/processed"
# 生成文件名
file_name = f"{processor.scenario.log_name}_{processor.scenario.token}_{processor.scenario.scenario_type}.pt"

# 确保文件名合法（移除特殊字符）
file_name = file_name.replace(":", "_").replace("/", "_").replace(" ", "_")

# 指定存放路径
os.makedirs(save_path, exist_ok=True)  # 确保目录存在

# 完整文件路径
file_path = os.path.join(save_path, file_name)
torch.save(data, file_path)


In [31]:
map_version = "nuplan-maps-v1.0"
data_path = "/share/data_cold/open_data/nuplan/data/cache/mini"
map_path = "/share/data_cold/open_data/nuplan/maps"
save_path = "/cailiu2/Diffusion-Planner/data/processed"
total_scenarios = 10
scenario_mapping = ScenarioMapping(scenario_map=get_scenario_map(), subsample_ratio_override=0.5)
builder = NuPlanScenarioBuilder(data_path, map_path, None, None, map_version, scenario_mapping = scenario_mapping)
worker = SingleMachineParallelExecutor(use_process_pool=True, max_workers = 128)
scenario_filter = ScenarioFilter(*get_filter_parameters(num_scenarios_per_type=10,
                                                          limit_total_scenarios=total_scenarios))
scenarios = builder.get_scenarios(scenario_filter, worker)
del worker, builder, scenario_filter
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
processor = DataProcessor(scenarios, device)
processor.work(save_path, debug=False)

Worker threads: 128


  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'DataProcessor' object has no attribute 'future_time_horizon'